In [16]:
import os
import string
import sys
from pathlib import Path
from os import PathLike

import string

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

import spacy
import spacy.lang.en

# visualization
import spacy.displacy

In [3]:
wine_reviews_file = Path.cwd() / 'data/winemag-data_first150k.csv'
reviews = pd.read_csv(str(wine_reviews_file))
reviews.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,variety,winery
0,0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz
1,1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez
2,2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley
3,3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi
4,4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,NaN,Provence red blend,Domaine de la Bégude


In [8]:
nlp = spacy.load('en_core_web_lg')

In [15]:
doc = nlp(reviews['description'][3])
spacy.displacy.render(doc, style='ent', jupyter=True)

In [17]:
punctuations = string.punctuation
stopwords = list(spacy.lang.en.stop_words.STOP_WORDS)

In [19]:
r = str(" ".join([i.lemma_ for i in doc]))
doc = nlp(r)
spacy.displacy.render(doc, style='ent', jupyter=True)

In [ ]:
# parsing reviews
parser = spacy.lang.en.English()
def spacy_tokenize(sentence: str):
    tokens = parser(sentence)
    tokens = [word.lemma_.strip().lower() if word.lemma_ != '-PRON-' else word.lower_ for word in tokens]